In [1]:
#Data Manipulation and handeling
import pandas as pd
import numpy as np

#Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sci

#Stats Libraries
from scipy import stats
from statsmodels.stats import weightstats as stests

#Multicollinearity Test and Teatment Libraries
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

#Data Preprocessing and EDA Libraries
from collections import OrderedDict
from sklearn.preprocessing import StandardScaler

#Model Selection Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, learning_curve

#ML Models
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
import xgboost
from xgboost import XGBRegressor

#Model Evaluation Libraries
from sklearn.metrics import r2_score, mean_squared_error

#Unsupervised ML library
from sklearn.cluster import KMeans


#Warning filter library
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Loading the dataset
supply_chain_train = pd.read_csv('Supplychain train dataset.csv')
supply_chain_test = pd.read_csv('Supply chain test dataset.csv')
supply_chain_train.head()
supply_chain_test.head()

,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
0,WH_122250,EID_72250,Rural,Large,North,Zone 6,7,0,2,5397,...,0,1,75,30.0,2019.0,9,1,A,1,29
1,WH_122251,EID_72251,Rural,Large,North,Zone 6,8,2,2,5183,...,0,0,138,20.0,2006.0,25,0,A,4,23
2,WH_122252,EID_72252,Rural,Large,North,Zone 6,3,0,4,4665,...,0,1,188,34.0,NaN,24,0,B,4,23
3,WH_122253,EID_72253,Rural,Mid,North,Zone 2,2,0,3,5196,...,0,1,156,31.0,NaN,15,0,A,4,1
4,WH_122254,EID_72254,Rural,Mid,North,Zone 2,5,2,4,8275,...,0,1,237,NaN,NaN,21,1,B+,2,12


In [3]:
supply_chain_train_copy=supply_chain_train
supply_chain_test_copy=supply_chain_test

In [4]:
supply_chain_train_copy.info()
supply_chain_test_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22150 entries, 0 to 22149
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ware_house_ID                 22150 non-null  object 
 1   WH_Manager_ID                 22150 non-null  object 
 2   Location_type                 22150 non-null  object 
 3   WH_capacity_size              22150 non-null  object 
 4   zone                          22150 non-null  object 
 5   WH_regional_zone              22150 non-null  object 
 6   num_refill_req_l3m            22150 non-null  int64  
 7   transport_issue_l1y           22150 non-null  int64  
 8   Competitor_in_mkt             22150 non-null  int64  
 9   retail_shop_num               22150 non-null  int64  
 10  wh_owner_type                 22150 non-null  object 
 11  distributor_num               22150 non-null  int64  
 12  flood_impacted                22150 non-null  int64  
 13  f

In [5]:
supply_chain_train_copy.shape


(22150, 24)

In [6]:
supply_chain_test_copy.shape

(2750, 23)

In [7]:
supply_chain_train_copy.describe().T

,count,mean,std,min,25%,50%,75%,max
num_refill_req_l3m,22150.0,4.097020,2.606289,0.0,2.00,4.0,6.0,8.0
transport_issue_l1y,22150.0,0.777201,1.201747,0.0,0.00,0.0,1.0,5.0
Competitor_in_mkt,22150.0,3.103928,1.142886,0.0,2.00,3.0,4.0,12.0
retail_shop_num,22150.0,4983.115711,1050.634225,1821.0,4309.25,4859.0,5499.0,11008.0
distributor_num,22150.0,42.386998,16.057730,15.0,29.00,42.0,56.0,70.0
flood_impacted,22150.0,0.098691,0.298253,0.0,0.00,0.0,0.0,1.0
flood_proof,22150.0,0.054492,0.226991,0.0,0.00,0.0,0.0,1.0
electric_supply,22150.0,0.656072,0.475028,0.0,0.00,1.0,1.0,1.0
dist_from_hub,22150.0,163.613725,62.660709,55.0,109.00,164.0,218.0,271.0
workers_num,21273.0,28.936398,7.843431,10.0,24.00,28.0,33.0,98.0


In [8]:
supply_chain_test_copy.describe().T

,count,mean,std,min,25%,50%,75%,max
num_refill_req_l3m,2750.0,4.030182,2.610002,0.0,2.00,4.0,6.0,8.0
transport_issue_l1y,2750.0,0.745818,1.183942,0.0,0.00,0.0,1.0,5.0
Competitor_in_mkt,2750.0,3.114909,1.134737,1.0,2.00,3.0,4.0,9.0
retail_shop_num,2750.0,5011.705818,1066.379543,2018.0,4341.25,4871.0,5504.0,10562.0
distributor_num,2750.0,42.655636,16.133231,15.0,29.00,43.0,57.0,70.0
flood_impacted,2750.0,0.096000,0.294645,0.0,0.00,0.0,0.0,1.0
flood_proof,2750.0,0.056000,0.229964,0.0,0.00,0.0,0.0,1.0
electric_supply,2750.0,0.662545,0.472927,0.0,0.00,1.0,1.0,1.0
dist_from_hub,2750.0,163.157818,63.162420,55.0,109.00,164.0,217.0,271.0
workers_num,2640.0,29.007197,8.057788,11.0,24.00,28.0,33.0,92.0


In [9]:
supply_chain_train_copy.isnull().sum()

Ware_house_ID                       0
WH_Manager_ID                       0
Location_type                       0
WH_capacity_size                    0
zone                                0
WH_regional_zone                    0
num_refill_req_l3m                  0
transport_issue_l1y                 0
Competitor_in_mkt                   0
retail_shop_num                     0
wh_owner_type                       0
distributor_num                     0
flood_impacted                      0
flood_proof                         0
electric_supply                     0
dist_from_hub                       0
workers_num                       877
wh_est_year                     10545
storage_issue_reported_l3m          0
temp_reg_mach                       0
approved_wh_govt_certificate      805
wh_breakdown_l3m                    0
govt_check_l3m                      0
product_wg_ton                      0
dtype: int64

In [10]:
supply_chain_test_copy.isnull().sum()

Ware_house_ID                      0
WH_Manager_ID                      0
Location_type                      0
WH_capacity_size                   0
zone                               0
WH_regional_zone                   0
num_refill_req_l3m                 0
transport_issue_l1y                0
Competitor_in_mkt                  0
retail_shop_num                    0
wh_owner_type                      0
distributor_num                    0
flood_impacted                     0
flood_proof                        0
electric_supply                    0
dist_from_hub                      0
workers_num                      110
wh_est_year                     1288
storage_issue_reported_l3m         0
temp_reg_mach                      0
approved_wh_govt_certificate     101
wh_breakdown_l3m                   0
govt_check_l3m                     0
dtype: int64

# OutLier detectopn and removal

In [11]:
def custom_summary(my_df):
    result = []
    for col in my_df.columns:
        if my_df[col].dtypes != 'object' and 'Categorical':
            stats = OrderedDict({
                'Feature Name': col , 
                'Count': my_df[col].count() ,
                'Minimum': my_df[col].min() ,
                'Quartile1': my_df[col].quantile(.25) ,
                'Quartile2': my_df[col].quantile(.50) ,
                'Mean': my_df[col].mean() ,
                'Quartile 3': my_df[col].quantile(.75) ,
                'Maximum': my_df[col].max() ,
                'Variance': round(my_df[col].var()) ,
                'Standard Deviation': my_df[col].std() ,
                'Skewness': my_df[col].skew() , 
                'Kurtosis': my_df[col].kurt()
                })
            result.append(stats)
    result_df = pd.DataFrame(result)
    #Skewness type: 
    skewness_label = []
    for i in result_df['Skewness']:
        if i <= -1:
            skewness_label.append('Highly Negatively Skewed')
        elif -1 < i <= -0.5:
            skewness_label.append('Moderately Negatively Skewed')
        elif -0.5 < i < 0:
            skewness_label.append('Fairly Negatively Skewed')
        elif 0 <= i < 0.5:
            skewness_label.append('Fairly Positively Skewed')
        elif 0.5 <= i < 1:
            skewness_label.append('Moderately Positively Skewed')
        elif i >= 1:
            skewness_label.append('Highly Positively Skewed')
    result_df['Skewness Comment'] = skewness_label
    kurtosis_label = []
    for i in result_df['Kurtosis']:
        if i >= 1:
            kurtosis_label.append('Leptokurtic Curve')
        elif i <= -1:
            kurtosis_label.append('Platykurtic Curve')
        else:
            kurtosis_label.append('Mesokurtic Curve')
    result_df['Kurtosis Comment'] = kurtosis_label
    Outliers_label = []
    for col in my_df.columns:
        if my_df[col].dtypes != 'object':
            Q1 = my_df[col].quantile(0.25)
            Q2 = my_df[col].quantile(0.5)
            Q3 = my_df[col].quantile(0.75)
            IQR = Q3 - Q1
            LW = Q1 - 1.5*IQR
            UW = Q3 + 1.5*IQR
            if len(my_df[(my_df[col] < LW) | (my_df[col] > UW)]) > 0:
                Outliers_label.append('Have Outliers')
            else:
                Outliers_label.append('No Outliers')
    result_df['Outlier Comment'] = Outliers_label

            
    return result_df

In [12]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
custom_summary(supply_chain_train_copy)


,Feature Name,Count,Minimum,Quartile1,Quartile2,Mean,Quartile 3,Maximum,Variance,Standard Deviation,Skewness,Kurtosis,Skewness Comment,Kurtosis Comment,Outlier Comment
0,num_refill_req_l3m,22150,0.00,2.00,4.00,4.10,6.00,8.00,7,2.61,-0.08,-1.22,Fairly Negatively Skewed,Platykurtic Curve,No Outliers
1,transport_issue_l1y,22150,0.00,0.00,0.00,0.78,1.00,5.00,1,1.20,1.61,1.82,Highly Positively Skewed,Leptokurtic Curve,Have Outliers
2,Competitor_in_mkt,22150,0.00,2.00,3.00,3.10,4.00,12.00,1,1.14,0.99,1.80,Moderately Positively Skewed,Leptokurtic Curve,Have Outliers
3,retail_shop_num,22150,1821.00,4309.25,4859.00,4983.12,5499.00,11008.00,1103832,1050.63,0.91,1.85,Moderately Positively Skewed,Leptokurtic Curve,Have Outliers
4,distributor_num,22150,15.00,29.00,42.00,42.39,56.00,70.00,258,16.06,0.02,-1.19,Fairly Positively Skewed,Platykurtic Curve,No Outliers
5,flood_impacted,22150,0.00,0.00,0.00,0.10,0.00,1.00,0,0.30,2.69,5.24,Highly Positively Skewed,Leptokurtic Curve,Have Outliers
6,flood_proof,22150,0.00,0.00,0.00,0.05,0.00,1.00,0,0.23,3.93,13.41,Highly Positively Skewed,Leptokurtic Curve,Have Outliers
7,electric_supply,22150,0.00,0.00,1.00,0.66,1.00,1.00,0,0.48,-0.66,-1.57,Moderately Negatively Skewed,Platykurtic Curve,No Outliers
8,dist_from_hub,22150,55.00,109.00,164.00,163.61,218.00,271.00,3926,62.66,-0.01,-1.20,Fairly Negatively Skewed,Platykurtic Curve,No Outliers
9,workers_num,21273,10.00,24.00,28.00,28.94,33.00,98.00,62,7.84,1.04,3.29,Highly Positively Skewed,Leptokurtic Curve,Have Outliers


In [13]:
custom_summary(supply_chain_test_copy)

,Feature Name,Count,Minimum,Quartile1,Quartile2,Mean,Quartile 3,Maximum,Variance,Standard Deviation,Skewness,Kurtosis,Skewness Comment,Kurtosis Comment,Outlier Comment
0,num_refill_req_l3m,2750,0.00,2.00,4.00,4.03,6.00,8.00,7,2.61,-0.03,-1.24,Fairly Negatively Skewed,Platykurtic Curve,No Outliers
1,transport_issue_l1y,2750,0.00,0.00,0.00,0.75,1.00,5.00,1,1.18,1.66,1.98,Highly Positively Skewed,Leptokurtic Curve,Have Outliers
2,Competitor_in_mkt,2750,1.00,2.00,3.00,3.11,4.00,9.00,1,1.13,0.91,1.62,Moderately Positively Skewed,Leptokurtic Curve,Have Outliers
3,retail_shop_num,2750,2018.00,4341.25,4871.00,5011.71,5504.00,10562.00,1137165,1066.38,0.97,1.92,Moderately Positively Skewed,Leptokurtic Curve,Have Outliers
4,distributor_num,2750,15.00,29.00,43.00,42.66,57.00,70.00,260,16.13,-0.01,-1.19,Fairly Negatively Skewed,Platykurtic Curve,No Outliers
5,flood_impacted,2750,0.00,0.00,0.00,0.10,0.00,1.00,0,0.29,2.74,5.54,Highly Positively Skewed,Leptokurtic Curve,Have Outliers
6,flood_proof,2750,0.00,0.00,0.00,0.06,0.00,1.00,0,0.23,3.86,12.94,Highly Positively Skewed,Leptokurtic Curve,Have Outliers
7,electric_supply,2750,0.00,0.00,1.00,0.66,1.00,1.00,0,0.47,-0.69,-1.53,Moderately Negatively Skewed,Platykurtic Curve,No Outliers
8,dist_from_hub,2750,55.00,109.00,164.00,163.16,217.00,271.00,3989,63.16,0.02,-1.20,Fairly Positively Skewed,Platykurtic Curve,No Outliers
9,workers_num,2640,11.00,24.00,28.00,29.01,33.00,92.00,65,8.06,1.15,4.04,Highly Positively Skewed,Leptokurtic Curve,Have Outliers


In [14]:
def replace_outlier(my_df , col , method = 'quartile',strategy = 'median'):
    col_data = my_df[col]
    
    if method == 'quartile':
        # Using quartile to calculate iqr
        q1 = my_df[col].quantile(.25)
        q2 = my_df[col].quantile(.50)
        q3 = my_df[col].quantile(.75)
        iqr = q3-q1
        lower_whisker = q1-1.5*iqr
        upper_whisker = q3+1.5*iqr
        
    elif method == 'standard deviation':
        mean = col_data.mean()
        std = col_data.std()
        lower_whisker = mean-2*std
        upper_whisker = mean+2*std
    else:
        print("Pass a correct method")
    
    # Printing all the outliers
    
    outliers = my_df.loc[(col_data < lower_whisker) | (col_data > upper_whisker)]
    outliers_density = round(len(outliers)/len(my_df),2)*100
    if len(outliers) == 0:
        print(f'Feature {col} does not have outliers')
        print("\n")
    else:
        print(f'Feature {col} has outliers')
        print("\n")
        print(f'Total number of outliers in {col} are {len(outliers)}')
        print("\n")
        print(f'Outlier percentage in {col} is {outliers_density}%')
        print("\n")
        display(my_df[(col_data < lower_whisker) | (col_data > upper_whisker)])
    
    # Replacing outliers
    if strategy == "median":
        my_df.loc[(col_data < lower_whisker) | (col_data > upper_whisker),col] = q2
    elif strategy == "mean":
        my_df.loc[(col_data < lower_whisker) | (col_data > upper_whisker),col] = mean
    else:
        print("Pass a correct strategy")
        
    return my_df

In [15]:
replace_outlier(supply_chain_train_copy,'transport_issue_l1y')
replace_outlier(supply_chain_train_copy,'Competitor_in_mkt')
replace_outlier(supply_chain_train_copy,'retail_shop_num')
replace_outlier(supply_chain_train_copy,'flood_impacted')
replace_outlier(supply_chain_train_copy,'workers_num')

Feature transport_issue_l1y has outliers


Total number of outliers in transport_issue_l1y are 2628


Outlier percentage in transport_issue_l1y is 12.0%




,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
3,WH_100003,EID_50003,Rural,Mid,North,Zone 3,7,4,2,6000,...,0,103,21.00,NaN,17,1,A+,3,27,22115
9,WH_100009,EID_50009,Rural,Small,South,Zone 6,4,3,3,3869,...,0,78,43.00,NaN,6,0,C,6,2,7130
26,WH_100026,EID_50026,Rural,Small,West,Zone 6,5,3,4,4063,...,1,229,21.00,NaN,8,0,C,6,9,10077
27,WH_100027,EID_50027,Rural,Small,South,Zone 6,8,3,2,5790,...,0,98,36.00,NaN,11,0,C,4,9,14150
57,WH_100057,EID_50057,Rural,Large,North,Zone 6,2,4,2,5308,...,1,77,39.00,NaN,17,0,B,3,23,19095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22080,WH_122080,EID_72080,Rural,Small,West,Zone 6,5,3,4,4620,...,1,89,34.00,2007.00,17,0,C,5,2,22131
22096,WH_122096,EID_72096,Rural,Mid,West,Zone 4,1,3,3,5187,...,1,57,30.00,NaN,15,0,B,3,26,18086
22113,WH_122113,EID_72113,Rural,Large,North,Zone 6,8,3,3,5204,...,0,70,33.00,NaN,6,1,B+,4,28,7121
22118,WH_122118,EID_72118,Rural,Small,North,Zone 1,3,5,4,4272,...,1,208,31.00,NaN,4,1,C,1,3,5103


Feature Competitor_in_mkt has outliers


Total number of outliers in Competitor_in_mkt are 85


Outlier percentage in Competitor_in_mkt is 0.0%




,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
37,WH_100037,EID_50037,Rural,Mid,East,Zone 3,0,1,8,6804,...,0,77,29.00,NaN,17,0,B,4,27,19100
127,WH_100127,EID_50127,Rural,Small,East,Zone 1,8,0,8,3948,...,1,161,25.00,2011.00,15,0,B,5,31,19124
296,WH_100296,EID_50296,Rural,Small,West,Zone 6,5,0,10,5025,...,1,160,21.00,2006.00,38,0,B,5,9,47145
352,WH_100352,EID_50352,Urban,Mid,West,Zone 2,3,0,8,4727,...,1,73,30.00,1998.00,25,0,C,4,12,32137
486,WH_100486,EID_50486,Rural,Mid,East,Zone 4,1,0,8,4212,...,0,79,16.00,NaN,27,0,C,2,26,33125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21010,WH_121010,EID_71010,Rural,Mid,East,Zone 4,2,0,8,3941,...,1,129,33.00,NaN,10,1,B+,2,26,13110
21065,WH_121065,EID_71065,Urban,Large,West,Zone 6,4,0,8,3748,...,0,158,50.00,NaN,11,1,B,3,23,13151
21202,WH_121202,EID_71202,Rural,Mid,East,Zone 4,2,0,8,5192,...,1,99,30.00,NaN,6,0,A,1,26,8139
21382,WH_121382,EID_71382,Rural,Small,West,Zone 6,6,1,8,7439,...,0,204,18.00,2012.00,20,0,C,2,8,26055


Feature retail_shop_num has outliers


Total number of outliers in retail_shop_num are 825


Outlier percentage in retail_shop_num is 4.0%




...

Feature flood_impacted has outliers


Total number of outliers in flood_impacted are 2186


Outlier percentage in flood_impacted is 10.0%




,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
4,WH_100004,EID_50004,Rural,Large,North,Zone 5,3,1,2,4740,...,1,112,25.00,2009.00,18,0,C,6,24,24071
16,WH_100016,EID_50016,Rural,Mid,West,Zone 4,0,0,2,5569,...,1,231,28.00,NaN,9,0,B+,5,11,12127
28,WH_100028,EID_50028,Rural,Small,North,Zone 6,3,1,1,4859,...,0,154,20.00,NaN,0,0,NaN,0,8,3123
58,WH_100058,EID_50058,Rural,Small,South,Zone 1,1,1,2,4004,...,1,168,27.00,NaN,25,1,C,4,21,32057
62,WH_100062,EID_50062,Rural,Large,South,Zone 6,7,0,4,4445,...,1,239,29.00,2012.00,10,1,A+,5,23,13061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22128,WH_122128,EID_72128,Rural,Large,North,Zone 5,3,0,3,4677,...,1,258,48.00,2002.00,33,0,A,5,24,45109
22129,WH_122129,EID_72129,Rural,Large,South,Zone 5,0,0,2,4113,...,0,143,NaN,NaN,8,0,C,3,10,11117
22134,WH_122134,EID_72134,Urban,Large,North,Zone 6,1,0,3,3837,...,1,87,25.00,NaN,12,0,A+,6,28,18114
22141,WH_122141,EID_72141,Rural,Mid,North,Zone 2,0,2,3,4844,...,1,211,34.00,NaN,4,0,B,2,18,5069


Feature workers_num has outliers


Total number of outliers in workers_num are 533


Outlier percentage in workers_num is 2.0%




,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
167,WH_100167,EID_50167,Rural,Mid,North,Zone 2,3,0,4,3972,...,1,152,47.00,2017.00,10,1,A+,1,12,15090
234,WH_100234,EID_50234,Rural,Large,West,Zone 5,8,2,4,4585,...,1,136,48.00,2005.00,17,1,A,6,25,22093
251,WH_100251,EID_50251,Rural,Mid,North,Zone 3,2,0,3,4859,...,0,76,50.00,NaN,37,1,A+,5,27,49091
266,WH_100266,EID_50266,Rural,Large,North,Zone 6,4,0,4,4943,...,1,183,62.00,2021.00,0,0,NaN,0,23,6123
282,WH_100282,EID_50282,Rural,Large,West,Zone 6,2,1,2,5045,...,0,148,49.00,NaN,15,0,B,5,23,18070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21992,WH_121992,EID_71992,Rural,Mid,North,Zone 4,4,0,6,3063,...,1,176,60.00,2007.00,39,0,B+,4,26,48122
21993,WH_121993,EID_71993,Urban,Large,North,Zone 5,2,1,2,4103,...,1,221,48.00,NaN,26,0,A+,3,10,33082
22110,WH_122110,EID_72110,Rural,Mid,South,Zone 2,0,0,3,4727,...,1,72,51.00,NaN,15,0,B+,2,12,17133
22111,WH_122111,EID_72111,Rural,Large,West,Zone 6,3,1,4,4608,...,1,244,72.00,NaN,9,0,A,6,23,11123


,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
0,WH_100000,EID_50000,Urban,Small,West,Zone 6,3,1,2,4651,...,1,91,29.00,NaN,13,0,A,5,15,17115
1,WH_100001,EID_50001,Rural,Large,North,Zone 5,0,0,4,6217,...,1,210,31.00,NaN,4,0,A,3,17,5074
2,WH_100002,EID_50002,Rural,Mid,South,Zone 2,1,0,4,4306,...,0,161,37.00,NaN,17,0,A,6,22,23137
3,WH_100003,EID_50003,Rural,Mid,North,Zone 3,7,0,2,6000,...,0,103,21.00,NaN,17,1,A+,3,27,22115
4,WH_100004,EID_50004,Rural,Large,North,Zone 5,3,1,2,4740,...,1,112,25.00,2009.00,18,0,C,6,24,24071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22145,WH_122145,EID_72145,Rural,Large,North,Zone 6,8,1,3,5030,...,1,203,28.00,2007.00,22,1,A+,2,4,29138
22146,WH_122146,EID_72146,Rural,Small,South,Zone 6,3,1,3,4320,...,0,170,NaN,NaN,26,0,C,5,9,33108
22147,WH_122147,EID_72147,Rural,Large,North,Zone 6,7,2,2,5268,...,0,99,22.00,NaN,20,0,B+,6,4,24072
22148,WH_122148,EID_72148,Rural,Large,North,Zone 6,6,2,4,4378,...,0,220,17.00,NaN,13,0,B,3,28,15055


In [16]:
replace_outlier(supply_chain_test_copy,'transport_issue_l1y')
replace_outlier(supply_chain_test_copy,'Competitor_in_mkt')
replace_outlier(supply_chain_test_copy,'retail_shop_num')
replace_outlier(supply_chain_test_copy,'flood_impacted')
replace_outlier(supply_chain_test_copy,'workers_num')

Feature transport_issue_l1y has outliers


Total number of outliers in transport_issue_l1y are 304


Outlier percentage in transport_issue_l1y is 11.0%




,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
9,WH_122259,EID_72259,Rural,Mid,North,Zone 2,4,3,4,4682,...,0,1,218,33.00,2002.00,25,1,C,3,12
10,WH_122260,EID_72260,Rural,Large,West,Zone 5,8,3,5,5468,...,1,1,71,33.00,NaN,9,1,A+,6,5
18,WH_122268,EID_72268,Rural,Large,North,Zone 5,1,5,3,5008,...,0,0,119,26.00,NaN,6,0,B,2,24
25,WH_122275,EID_72275,Urban,Mid,North,Zone 3,3,3,4,3354,...,0,1,209,28.00,NaN,12,0,C,1,27
28,WH_122278,EID_72278,Rural,Mid,West,Zone 4,4,3,2,7246,...,0,1,228,36.00,NaN,4,1,C,3,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,WH_124959,EID_74959,Rural,Large,North,Zone 5,8,4,4,6147,...,0,0,144,33.00,2000.00,19,1,A+,4,17
2710,WH_124960,EID_74960,Rural,Mid,North,Zone 4,1,3,3,3517,...,0,0,103,17.00,NaN,10,0,A+,6,26
2726,WH_124976,EID_74976,Rural,Large,North,Zone 6,8,4,3,4550,...,1,0,169,NaN,2009.00,15,1,A+,3,6
2728,WH_124978,EID_74978,Rural,Mid,South,Zone 3,0,3,3,4343,...,0,0,136,21.00,NaN,5,0,A,1,19


Feature Competitor_in_mkt has outliers


Total number of outliers in Competitor_in_mkt are 10


Outlier percentage in Competitor_in_mkt is 0.0%




,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
140,WH_122390,EID_72390,Rural,Mid,East,Zone 3,3,0,8,8571,...,0,1,64,38.00,2002.00,28,0,B,6,27
438,WH_122688,EID_72688,Rural,Mid,South,Zone 3,0,1,8,7998,...,0,1,96,35.00,NaN,22,0,B+,5,27
863,WH_123113,EID_73113,Rural,Mid,East,Zone 3,0,1,8,4784,...,0,0,133,24.00,NaN,17,0,C,4,27
984,WH_123234,EID_73234,Rural,Small,East,Zone 1,6,0,8,4613,...,0,0,265,16.00,2014.00,14,1,B+,3,31
1193,WH_123443,EID_73443,Rural,Mid,West,Zone 4,2,0,9,4232,...,0,1,193,33.00,NaN,11,0,A+,3,26
1233,WH_123483,EID_73483,Rural,Small,East,Zone 6,6,1,8,5178,...,0,1,248,33.00,NaN,11,0,B+,4,15
1376,WH_123626,EID_73626,Rural,Mid,West,Zone 2,1,0,9,3064,...,0,1,134,30.00,NaN,5,0,C,6,12
1807,WH_124057,EID_74057,Rural,Mid,East,Zone 4,1,1,8,2932,...,0,1,203,35.00,NaN,11,1,B,5,26
1853,WH_124103,EID_74103,Rural,Mid,West,Zone 3,0,0,9,4096,...,0,1,83,36.00,NaN,18,0,C,3,27
2012,WH_124262,EID_74262,Rural,Small,West,Zone 1,4,0,8,6268,...,0,1,201,33.00,2017.00,14,1,A,5,3


Feature retail_shop_num has outliers


Total number of outliers in retail_shop_num are 115


Outlier percentage in retail_shop_num is 4.0%




...

Feature flood_impacted has outliers


Total number of outliers in flood_impacted are 264


Outlier percentage in flood_impacted is 10.0%




,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
37,WH_122287,EID_72287,Rural,Small,South,Zone 6,7,1,3,4329,...,0,1,205,36.00,2015.00,10,1,C,3,2
40,WH_122290,EID_72290,Rural,Large,West,Zone 6,3,1,4,4315,...,0,1,266,39.00,NaN,7,0,C,4,23
51,WH_122301,EID_72301,Rural,Mid,West,Zone 2,0,0,4,5191,...,0,1,232,34.00,NaN,38,0,A+,4,12
60,WH_122310,EID_72310,Rural,Small,West,Zone 6,0,1,3,4791,...,0,1,111,37.00,NaN,34,0,A,6,2
72,WH_122322,EID_72322,Rural,Mid,West,Zone 4,6,0,4,4038,...,0,1,252,37.00,2000.00,28,1,A+,6,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2707,WH_124957,EID_74957,Urban,Mid,West,Zone 3,4,0,6,5092,...,0,1,182,NaN,2007.00,28,1,A+,5,19
2732,WH_124982,EID_74982,Rural,Mid,West,Zone 4,7,1,6,4347,...,1,1,78,32.00,1996.00,30,0,B+,6,26
2736,WH_124986,EID_74986,Rural,Small,West,Zone 6,6,0,2,4607,...,0,1,87,25.00,NaN,5,1,A+,4,8
2737,WH_124987,EID_74987,Rural,Large,West,Zone 6,4,0,4,4871,...,0,1,185,39.00,2018.00,12,1,B+,6,23


Feature workers_num has outliers


Total number of outliers in workers_num are 71


Outlier percentage in workers_num is 3.0%




,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
17,WH_122267,EID_72267,Rural,Small,West,Zone 6,7,0,3,4456,...,0,1,70,54.00,2016.00,12,0,C,1,8
107,WH_122357,EID_72357,Rural,Mid,North,Zone 3,3,0,3,6550,...,0,1,132,53.00,2016.00,8,0,C,3,27
108,WH_122358,EID_72358,Urban,Large,West,Zone 5,5,2,4,3654,...,0,1,61,47.00,2021.00,5,0,A,1,10
122,WH_122372,EID_72372,Rural,Mid,North,Zone 4,3,0,2,5730,...,0,1,150,52.00,2019.00,5,0,C,1,26
232,WH_122482,EID_72482,Rural,Large,North,Zone 6,2,2,3,6930,...,0,0,102,52.00,NaN,9,0,B+,1,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520,WH_124770,EID_74770,Rural,Large,North,Zone 5,5,1,3,7068,...,0,1,219,61.00,2015.00,11,0,A,3,17
2530,WH_124780,EID_74780,Urban,Mid,South,Zone 3,6,0,3,4175,...,0,1,230,55.00,2007.00,19,0,C,6,19
2652,WH_124902,EID_74902,Rural,Mid,West,Zone 2,0,0,2,5140,...,0,1,141,92.00,NaN,0,0,NaN,0,12
2687,WH_124937,EID_74937,Rural,Large,North,Zone 5,7,0,2,4242,...,0,1,235,50.00,2017.00,12,0,C,6,17


,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
0,WH_122250,EID_72250,Rural,Large,North,Zone 6,7,0,2,5397,...,0,1,75,30.00,2019.00,9,1,A,1,29
1,WH_122251,EID_72251,Rural,Large,North,Zone 6,8,2,2,5183,...,0,0,138,20.00,2006.00,25,0,A,4,23
2,WH_122252,EID_72252,Rural,Large,North,Zone 6,3,0,4,4665,...,0,1,188,34.00,NaN,24,0,B,4,23
3,WH_122253,EID_72253,Rural,Mid,North,Zone 2,2,0,3,5196,...,0,1,156,31.00,NaN,15,0,A,4,1
4,WH_122254,EID_72254,Rural,Mid,North,Zone 2,5,2,4,4871,...,0,1,237,NaN,NaN,21,1,B+,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,WH_124995,EID_74995,Rural,Small,North,Zone 1,3,0,4,5390,...,0,1,142,34.00,2005.00,22,1,A,2,30
2746,WH_124996,EID_74996,Rural,Mid,West,Zone 2,6,0,4,4490,...,0,1,130,28.00,2012.00,10,0,B,4,18
2747,WH_124997,EID_74997,Urban,Large,South,Zone 5,7,0,2,5403,...,0,1,147,NaN,NaN,23,0,B+,5,25
2748,WH_124998,EID_74998,Rural,Small,North,Zone 1,1,0,2,4871,...,0,1,60,25.00,NaN,18,0,A,6,30


# Missing values

In [17]:
supply_chain_train_copy['workers_num'].fillna(supply_chain_train_copy['workers_num'].median(), inplace = True)

supply_chain_test_copy['workers_num'].fillna(supply_chain_test_copy['workers_num'].median(), inplace = True)

# Merging of Labels in Columns

In [18]:
supply_chain_train_copy.drop(['Ware_house_ID','WH_Manager_ID'],axis=1,inplace=True)
supply_chain_test_copy.drop(['Ware_house_ID','WH_Manager_ID'],axis=1,inplace=True)

In [19]:
supply_chain_train_copy_category = pd.DataFrame(supply_chain_train_copy, columns=['Location_type', 'WH_capacity_size', 'zone','WH_regional_zone','wh_owner_type',
                                                        'approved_wh_govt_certificate'])
supply_chain_train_copy_category.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,wh_owner_type,approved_wh_govt_certificate
0,Urban,Small,West,Zone 6,Rented,A
1,Rural,Large,North,Zone 5,Company Owned,A
2,Rural,Mid,South,Zone 2,Company Owned,A
3,Rural,Mid,North,Zone 3,Rented,A+
4,Rural,Large,North,Zone 5,Company Owned,C


In [20]:
supply_chain_test_copy_category = pd.DataFrame(supply_chain_test_copy, columns=['Location_type', 'WH_capacity_size', 'zone','WH_regional_zone','wh_owner_type',
                                                        'approved_wh_govt_certificate'])
supply_chain_test_copy_category.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,wh_owner_type,approved_wh_govt_certificate
0,Rural,Large,North,Zone 6,Company Owned,A
1,Rural,Large,North,Zone 6,Rented,A
2,Rural,Large,North,Zone 6,Rented,B
3,Rural,Mid,North,Zone 2,Rented,A
4,Rural,Mid,North,Zone 2,Rented,B+


In [21]:
for col in supply_chain_train_copy_category.columns:
        print(col, ':' ,len(supply_chain_train_copy_category[col].unique()), 'labels')

Location_type : 2 labels
WH_capacity_size : 3 labels
zone : 4 labels
WH_regional_zone : 6 labels
wh_owner_type : 2 labels
approved_wh_govt_certificate : 6 labels


In [22]:
for col in supply_chain_test_copy_category.columns:
        print(col, ':' ,len(supply_chain_test_copy_category[col].unique()), 'labels')

Location_type : 2 labels
WH_capacity_size : 3 labels
zone : 4 labels
WH_regional_zone : 6 labels
wh_owner_type : 2 labels
approved_wh_govt_certificate : 6 labels


# Merging of Labels in the columns(Grouping of Labels)

In [23]:
def merge_labels(df, column, labels_to_merge, new_label):
    df[column] = df[column].replace(labels_to_merge, new_label)
    return df

In [24]:
supply_chain_train_copy=merge_labels(supply_chain_train_copy,'WH_capacity_size',['Large','Mid'],'Large')
supply_chain_train_copy=merge_labels(supply_chain_train_copy,'approved_wh_govt_certificate',['B+','B'],'B')
supply_chain_train_copy=merge_labels(supply_chain_train_copy,'approved_wh_govt_certificate',['A+','A'],'A')

In [25]:
supply_chain_test_copy=merge_labels(supply_chain_test_copy,'WH_capacity_size',['Large','Mid'],'Large')
supply_chain_test_copy=merge_labels(supply_chain_test_copy,'approved_wh_govt_certificate',['B+','B'],'B')
supply_chain_test_copy=merge_labels(supply_chain_test_copy,'approved_wh_govt_certificate',['A+','A'],'A')

# Label Encoding

In [26]:
supply_chain_train_copy.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,wh_owner_type,distributor_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
0,Urban,Small,West,Zone 6,3,1,2,4651,Rented,24,...,1,91,29.00,NaN,13,0,A,5,15,17115
1,Rural,Large,North,Zone 5,0,0,4,6217,Company Owned,47,...,1,210,31.00,NaN,4,0,A,3,17,5074
2,Rural,Large,South,Zone 2,1,0,4,4306,Company Owned,64,...,0,161,37.00,NaN,17,0,A,6,22,23137
3,Rural,Large,North,Zone 3,7,0,2,6000,Rented,50,...,0,103,21.00,NaN,17,1,A,3,27,22115
4,Rural,Large,North,Zone 5,3,1,2,4740,Company Owned,42,...,1,112,25.00,2009.00,18,0,C,6,24,24071


In [27]:
supply_chain_test_copy.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,wh_owner_type,distributor_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
0,Rural,Large,North,Zone 6,7,0,2,5397,Company Owned,26,...,0,1,75,30.00,2019.00,9,1,A,1,29
1,Rural,Large,North,Zone 6,8,2,2,5183,Rented,61,...,0,0,138,20.00,2006.00,25,0,A,4,23
2,Rural,Large,North,Zone 6,3,0,4,4665,Rented,26,...,0,1,188,34.00,NaN,24,0,B,4,23
3,Rural,Large,North,Zone 2,2,0,3,5196,Rented,36,...,0,1,156,31.00,NaN,15,0,A,4,1
4,Rural,Large,North,Zone 2,5,2,4,4871,Rented,58,...,0,1,237,28.00,NaN,21,1,B,2,12


In [28]:
from sklearn.preprocessing import LabelEncoder
def label_encode_columns(df, columns):
    label_encoder = LabelEncoder()
    for column in columns:
        if df[column].dtype == 'object':
            df[column] = label_encoder.fit_transform(df[column])
    return df

In [29]:
# Replace 'unknown' with NaN

columns = ['Location_type','WH_capacity_size', 'zone', 'WH_regional_zone', 'wh_owner_type','approved_wh_govt_certificate']
supply_chain_train_copy = label_encode_columns(supply_chain_train_copy, columns)
supply_chain_train_copy['wh_est_year'] = supply_chain_train_copy['wh_est_year'].replace('Unknown', np.nan)
supply_chain_train_copy['wh_est_year'] = supply_chain_train_copy['wh_est_year'].fillna(method='backfill')
supply_chain_train_copy['wh_est_year'] = supply_chain_train_copy['wh_est_year'].fillna(method='ffill')

supply_chain_train_copy['wh_est_year']=supply_chain_train_copy['wh_est_year'].astype(int)
supply_chain_train_copy.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,wh_owner_type,distributor_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
0,1,1,3,5,3,1,2,4651,1,24,...,1,91,29.00,2009,13,0,0,5,15,17115
1,0,0,1,4,0,0,4,6217,0,47,...,1,210,31.00,2009,4,0,0,3,17,5074
2,0,0,2,1,1,0,4,4306,0,64,...,0,161,37.00,2009,17,0,0,6,22,23137
3,0,0,1,2,7,0,2,6000,1,50,...,0,103,21.00,2009,17,1,0,3,27,22115
4,0,0,1,4,3,1,2,4740,0,42,...,1,112,25.00,2009,18,0,2,6,24,24071


In [30]:
# Replace 'unknown' with NaN

columns = ['Location_type','WH_capacity_size', 'zone', 'WH_regional_zone', 'wh_owner_type','approved_wh_govt_certificate']
supply_chain_test_copy = label_encode_columns(supply_chain_test_copy , columns)
supply_chain_test_copy['wh_est_year'] = supply_chain_test_copy['wh_est_year'].replace('Unknown', np.nan)
supply_chain_test_copy ['wh_est_year'] = supply_chain_test_copy['wh_est_year'].fillna(method='backfill')
supply_chain_test_copy['wh_est_year'] = supply_chain_test_copy['wh_est_year'].fillna(method='ffill')

supply_chain_test_copy['wh_est_year']=supply_chain_test_copy['wh_est_year'].astype(int)
supply_chain_test_copy.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,wh_owner_type,distributor_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
0,0,0,1,5,7,0,2,5397,0,26,...,0,1,75,30.00,2019,9,1,0,1,29
1,0,0,1,5,8,2,2,5183,1,61,...,0,0,138,20.00,2006,25,0,0,4,23
2,0,0,1,5,3,0,4,4665,1,26,...,0,1,188,34.00,2004,24,0,1,4,23
3,0,0,1,1,2,0,3,5196,1,36,...,0,1,156,31.00,2004,15,0,0,4,1
4,0,0,1,1,5,2,4,4871,1,58,...,0,1,237,28.00,2004,21,1,1,2,12


# Min Max Scaler

In [31]:
from sklearn.preprocessing import MinMaxScaler
# Select the features to apply scaling
train_features = ['Location_type','WH_capacity_size','zone','WH_regional_zone', 'num_refill_req_l3m', 'transport_issue_l1y',
            'Competitor_in_mkt', 'retail_shop_num', 'wh_owner_type', 'distributor_num', 'flood_impacted',
            'flood_proof', 'electric_supply', 'dist_from_hub', 'workers_num', 'wh_est_year',
            'storage_issue_reported_l3m', 'temp_reg_mach', 'approved_wh_govt_certificate',
            'wh_breakdown_l3m', 'govt_check_l3m','product_wg_ton']

# Apply MinMaxScaler to numeric features
scaler = MinMaxScaler()
supply_chain_train_copy = supply_chain_train_copy[train_features].select_dtypes(include=['int64', 'float64'])
supply_chain_train_copy = pd.DataFrame(scaler.fit_transform(supply_chain_train_copy), columns=supply_chain_train_copy.columns)


# Combine the scaled numeric and encoded categorical features
supply_chain_train_copy_transformed = pd.concat([supply_chain_train_copy])

In [32]:
supply_chain_train_copy_transformed.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,wh_owner_type,distributor_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
0,1.00,1.00,1.00,1.00,0.38,0.50,0.29,0.45,1.00,0.16,...,1.00,0.17,0.51,0.48,0.33,0.00,0.00,0.83,0.45,0.28
1,0.00,0.00,0.33,0.80,0.00,0.00,0.57,0.78,0.00,0.58,...,1.00,0.72,0.57,0.48,0.10,0.00,0.00,0.50,0.52,0.06
2,0.00,0.00,0.67,0.20,0.12,0.00,0.57,0.37,0.00,0.89,...,0.00,0.49,0.74,0.48,0.44,0.00,0.00,1.00,0.68,0.40
3,0.00,0.00,0.33,0.40,0.88,0.00,0.29,0.73,1.00,0.64,...,0.00,0.22,0.29,0.48,0.44,1.00,0.00,0.50,0.84,0.38
4,0.00,0.00,0.33,0.80,0.38,0.50,0.29,0.47,0.00,0.49,...,1.00,0.26,0.40,0.48,0.46,0.00,0.67,1.00,0.74,0.41


In [33]:
supply_chain_train_copy_transformed.columns

Index(['Location_type', 'WH_capacity_size', 'zone', 'WH_regional_zone',
       'num_refill_req_l3m', 'transport_issue_l1y', 'Competitor_in_mkt',
       'retail_shop_num', 'wh_owner_type', 'distributor_num', 'flood_impacted',
       'flood_proof', 'electric_supply', 'dist_from_hub', 'workers_num',
       'wh_est_year', 'storage_issue_reported_l3m', 'temp_reg_mach',
       'approved_wh_govt_certificate', 'wh_breakdown_l3m', 'govt_check_l3m',
       'product_wg_ton'],
      dtype='object')

In [34]:
test_features = ['Location_type','WH_capacity_size','zone','WH_regional_zone', 'num_refill_req_l3m', 'transport_issue_l1y',
            'Competitor_in_mkt', 'retail_shop_num', 'wh_owner_type', 'distributor_num', 'flood_impacted',
            'flood_proof', 'electric_supply', 'dist_from_hub', 'workers_num', 'wh_est_year',
            'storage_issue_reported_l3m', 'temp_reg_mach', 'approved_wh_govt_certificate',
            'wh_breakdown_l3m', 'govt_check_l3m']


# Apply MinMaxScaler to numeric features
scaler = MinMaxScaler()
supply_chain_test_copy = supply_chain_test_copy[test_features].select_dtypes(include=['int64', 'float64'])
supply_chain_test_copy = pd.DataFrame(scaler.fit_transform(supply_chain_test_copy), columns=supply_chain_test_copy.columns)


# Combine the scaled numeric and encoded categorical features
supply_chain_test_copy_transformed = pd.concat([supply_chain_test_copy])

In [35]:
supply_chain_test_copy_transformed.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,wh_owner_type,distributor_num,...,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m
0,0.00,0.00,0.33,1.00,0.88,0.00,0.17,0.60,0.00,0.20,...,0.00,1.00,0.09,0.54,0.85,0.23,1.00,0.00,0.17,0.90
1,0.00,0.00,0.33,1.00,1.00,1.00,0.17,0.55,1.00,0.84,...,0.00,0.00,0.38,0.26,0.37,0.64,0.00,0.00,0.67,0.71
2,0.00,0.00,0.33,1.00,0.38,0.00,0.50,0.44,1.00,0.20,...,0.00,1.00,0.62,0.66,0.30,0.62,0.00,0.33,0.67,0.71
3,0.00,0.00,0.33,0.20,0.25,0.00,0.33,0.56,1.00,0.38,...,0.00,1.00,0.47,0.57,0.30,0.38,0.00,0.00,0.67,0.00
4,0.00,0.00,0.33,0.20,0.62,1.00,0.50,0.49,1.00,0.78,...,0.00,1.00,0.84,0.49,0.30,0.54,1.00,0.33,0.33,0.35


# PCA

# Collinearity with target feature

In [36]:
def correlation_with_target(data,t_col):
    independent_variable = data.drop(t_col,axis = 1).columns # axis =1 t_col Column will be drop
    corr_result = []
    for col in independent_variable:
        corr_result.append(data[t_col].corr(data[col]))
    result = pd.DataFrame([independent_variable,corr_result],index=['Independent variables','Correlation']).T #Transpose
    return result.sort_values('Correlation',ascending = False)

In [37]:
correlation_with_target(supply_chain_train_copy_transformed,'product_wg_ton')

,Independent variables,Correlation
16,storage_issue_reported_l3m,0.99
19,wh_breakdown_l3m,0.34
17,temp_reg_mach,0.10
0,Location_type,0.07
6,Competitor_in_mkt,0.01
9,distributor_num,0.01
8,wh_owner_type,0.00
4,num_refill_req_l3m,0.00
3,WH_regional_zone,0.00
12,electric_supply,-0.00


In [38]:
supply_chain_train_copy_transformed.isnull().sum()

Location_type                   0
WH_capacity_size                0
zone                            0
WH_regional_zone                0
num_refill_req_l3m              0
transport_issue_l1y             0
Competitor_in_mkt               0
retail_shop_num                 0
wh_owner_type                   0
distributor_num                 0
flood_impacted                  0
flood_proof                     0
electric_supply                 0
dist_from_hub                   0
workers_num                     0
wh_est_year                     0
storage_issue_reported_l3m      0
temp_reg_mach                   0
approved_wh_govt_certificate    0
wh_breakdown_l3m                0
govt_check_l3m                  0
product_wg_ton                  0
dtype: int64

In [39]:
supply_chain_test_copy_transformed.isnull().sum()

Location_type                   0
WH_capacity_size                0
zone                            0
WH_regional_zone                0
num_refill_req_l3m              0
transport_issue_l1y             0
Competitor_in_mkt               0
retail_shop_num                 0
wh_owner_type                   0
distributor_num                 0
flood_impacted                  0
flood_proof                     0
electric_supply                 0
dist_from_hub                   0
workers_num                     0
wh_est_year                     0
storage_issue_reported_l3m      0
temp_reg_mach                   0
approved_wh_govt_certificate    0
wh_breakdown_l3m                0
govt_check_l3m                  0
dtype: int64

In [40]:
supply_chain_train_copy_transformed['workers_num'].fillna(supply_chain_train_copy_transformed['workers_num'].median(), inplace = True)

In [41]:
def custom_PCA(X):
    n_comp = len(X.columns)
    
# Applying StandardScaler for feature scaling (Feature Scaling Technique)
    sc = StandardScaler()
    X = sc.fit_transform(X)
    
    
    for i in range(1,n_comp):
        pca = PCA(n_components = i)
        p_comp = pca.fit_transform(X)
        evr = np.cumsum(pca.explained_variance_ratio_)
        if evr[i-1] > 0.9: # Means 90% of the information
            pcs = i
            break
    print("Explained Variance Ratio after PCA is : ", evr)
    # Creating a Dataframes of Principal Components
    col = []
    for j in range(1,pcs+1):
        col.append("PC_"+str(j))
    pca_df = pd.DataFrame(p_comp,columns = col)
    return pca_df
     

In [42]:
supply_chain_train_copy_transformed.columns

Index(['Location_type', 'WH_capacity_size', 'zone', 'WH_regional_zone',
       'num_refill_req_l3m', 'transport_issue_l1y', 'Competitor_in_mkt',
       'retail_shop_num', 'wh_owner_type', 'distributor_num', 'flood_impacted',
       'flood_proof', 'electric_supply', 'dist_from_hub', 'workers_num',
       'wh_est_year', 'storage_issue_reported_l3m', 'temp_reg_mach',
       'approved_wh_govt_certificate', 'wh_breakdown_l3m', 'govt_check_l3m',
       'product_wg_ton'],
      dtype='object')

In [43]:
supply_chain_train_copy_transformed['flood_proof']

0       1.00
1       0.00
2       0.00
3       0.00
4       0.00
        ... 
22145   0.00
22146   0.00
22147   0.00
22148   0.00
22149   0.00
Name: flood_proof, Length: 22150, dtype: float64

In [44]:
pca_df = custom_PCA(supply_chain_train_copy_transformed.drop("product_wg_ton",axis = 1))

Explained Variance Ratio after PCA is :  [0.09667939 0.17501323 0.24408183 0.30834987 0.36902994 0.42262367
 0.4741015  0.52405642 0.5736531  0.62284533 0.6719264  0.71790779
 0.76316513 0.80496196 0.8461863  0.8843919  0.91610638]


In [45]:
supply_chain_train_copy_transformed.head()

,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,wh_owner_type,distributor_num,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
0,1.00,1.00,1.00,1.00,0.38,0.50,0.29,0.45,1.00,0.16,...,1.00,0.17,0.51,0.48,0.33,0.00,0.00,0.83,0.45,0.28
1,0.00,0.00,0.33,0.80,0.00,0.00,0.57,0.78,0.00,0.58,...,1.00,0.72,0.57,0.48,0.10,0.00,0.00,0.50,0.52,0.06
2,0.00,0.00,0.67,0.20,0.12,0.00,0.57,0.37,0.00,0.89,...,0.00,0.49,0.74,0.48,0.44,0.00,0.00,1.00,0.68,0.40
3,0.00,0.00,0.33,0.40,0.88,0.00,0.29,0.73,1.00,0.64,...,0.00,0.22,0.29,0.48,0.44,1.00,0.00,0.50,0.84,0.38
4,0.00,0.00,0.33,0.80,0.38,0.50,0.29,0.47,0.00,0.49,...,1.00,0.26,0.40,0.48,0.46,0.00,0.67,1.00,0.74,0.41


In [46]:
pca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22150 entries, 0 to 22149
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC_1    22150 non-null  float64
 1   PC_2    22150 non-null  float64
 2   PC_3    22150 non-null  float64
 3   PC_4    22150 non-null  float64
 4   PC_5    22150 non-null  float64
 5   PC_6    22150 non-null  float64
 6   PC_7    22150 non-null  float64
 7   PC_8    22150 non-null  float64
 8   PC_9    22150 non-null  float64
 9   PC_10   22150 non-null  float64
 10  PC_11   22150 non-null  float64
 11  PC_12   22150 non-null  float64
 12  PC_13   22150 non-null  float64
 13  PC_14   22150 non-null  float64
 14  PC_15   22150 non-null  float64
 15  PC_16   22150 non-null  float64
 16  PC_17   22150 non-null  float64
dtypes: float64(17)
memory usage: 2.9 MB


In [47]:
pca_df = pca_df.join(supply_chain_train_copy["product_wg_ton"],how = "left") # left join using

In [48]:
pca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22150 entries, 0 to 22149
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PC_1            22150 non-null  float64
 1   PC_2            22150 non-null  float64
 2   PC_3            22150 non-null  float64
 3   PC_4            22150 non-null  float64
 4   PC_5            22150 non-null  float64
 5   PC_6            22150 non-null  float64
 6   PC_7            22150 non-null  float64
 7   PC_8            22150 non-null  float64
 8   PC_9            22150 non-null  float64
 9   PC_10           22150 non-null  float64
 10  PC_11           22150 non-null  float64
 11  PC_12           22150 non-null  float64
 12  PC_13           22150 non-null  float64
 13  PC_14           22150 non-null  float64
 14  PC_15           22150 non-null  float64
 15  PC_16           22150 non-null  float64
 16  PC_17           22150 non-null  float64
 17  product_wg_ton  22150 non-null 

In [49]:
pca_df_test = custom_PCA(supply_chain_test_copy_transformed)

Explained Variance Ratio after PCA is :  [0.10033652 0.17989912 0.25069492 0.31213733 0.37172792 0.42849462
 0.48311909 0.53521718 0.58547467 0.63396363 0.68043912 0.72451304
 0.76708624 0.80839098 0.84820161 0.88513399 0.91752215]


In [50]:
pca_df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2750 entries, 0 to 2749
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC_1    2750 non-null   float64
 1   PC_2    2750 non-null   float64
 2   PC_3    2750 non-null   float64
 3   PC_4    2750 non-null   float64
 4   PC_5    2750 non-null   float64
 5   PC_6    2750 non-null   float64
 6   PC_7    2750 non-null   float64
 7   PC_8    2750 non-null   float64
 8   PC_9    2750 non-null   float64
 9   PC_10   2750 non-null   float64
 10  PC_11   2750 non-null   float64
 11  PC_12   2750 non-null   float64
 12  PC_13   2750 non-null   float64
 13  PC_14   2750 non-null   float64
 14  PC_15   2750 non-null   float64
 15  PC_16   2750 non-null   float64
 16  PC_17   2750 non-null   float64
dtypes: float64(17)
memory usage: 365.4 KB


# Model Building

In [51]:
X_train=pca_df.drop('product_wg_ton' ,axis=1)
y_train=pca_df['product_wg_ton']
X_test=pca_df_test

In [53]:
# Model Evaluation
from sklearn.metrics import r2_score,mean_squared_error
from sklearn import metrics
from sklearn.metrics import mean_absolute_percentage_error
import math


X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('y_pred is{} '.format(y_pred))
print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))


y_pred is[0.43694045 0.65184278 0.19878295 ... 0.5746372  0.50512381 0.52324054] 
MAE is 0.09580046444051402
MAPE is 0.5331772603793099
MSE is 0.01550865958001249
R2 score is 0.6767399909809941
RMSE score is 0.12453376883404955


As the performance metrics is not so graet we will try some other regression models

# Lasso

In [64]:

X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model = Lasso()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))

MAE is 0.18076003874759805
MAPE is 147.87898524622312
MSE is 0.04797581621878584
R2 score is -2.0656629740045673e-07
RMSE score is 0.21903382437145602


# Ridge

In [65]:

X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model = Ridge()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))



MAE is 0.09580050165853315
MAPE is 53.31789609685731
MSE is 0.01550866449022182
R2 score is 0.6767398886333797
RMSE score is 0.12453378854841693


# DTR

In [67]:
X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model =DecisionTreeRegressor()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))

MAE is 0.12298370317630175
MAPE is 52.58716221020441
MSE is 0.027061169249809707
R2 score is 0.4359413351858946
RMSE score is 0.16450279404864135


# KNR

In [68]:
X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model =KNeighborsRegressor()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))

MAE is 0.11051798397914572
MAPE is 55.71330678608201
MSE is 0.020590031383630154
R2 score is 0.5708246933634392
RMSE score is 0.14349226942114393


# RandomForestRegressor

In [54]:
X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model =RandomForestRegressor()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('y_pred is{} '.format(y_pred))
print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))

y_pred is[0.3912282  0.68825679 0.15236804 ... 0.60598143 0.51476416 0.50918943] 
MAE is 0.0864145852944951
MAPE is 0.4060144562024283
MSE is 0.0131412563012539
R2 score is 0.726085764630538
RMSE score is 0.11463531873403546


In [ ]:
imp_df = pd.DataFrame({
    "Feature Name": X_train.columns,
    "Importance": model.feature_importances_
})
fi = imp_df.sort_values(by="Importance", ascending=False)

fi2 = fi.head(10)
plt.figure(figsize=(10,8))
sns.barplot(data=fi2, x='Importance', y='Feature Name')
plt.title('Feature Importance Each Attributes (Random Forest Regressor)', fontsize=18)
plt.xlabel ('Importance', fontsize=16)
plt.ylabel ('Feature Name', fontsize=16)
plt.show()

# AdaBoostRegressor

In [71]:
X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model =AdaBoostRegressor()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))

MAE is 0.11216179051647962
MAPE is 90.84386355916452
MSE is 0.018836460157700876
R2 score is 0.6073758503080364
RMSE score is 0.1372459841223082


In [72]:
X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model =GradientBoostingRegressor()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))

MAE is 0.08829843149684555
MAPE is 46.527245612641934
MSE is 0.013504262004907402
R2 score is 0.7185193320557837
RMSE score is 0.11620783968780851


# XGBRegressor

In [55]:
X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model =XGBRegressor()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('y_pred is{} '.format(y_pred))
print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))

y_pred is[0.3313722  0.70371747 0.16653189 ... 0.64893556 0.5161298  0.514451  ] 
MAE is 0.08030466978614587
MAPE is 0.39923278343915897
MSE is 0.011222912429214093
R2 score is 0.7660714161420559
RMSE score is 0.10593824818833891


# SVR

In [56]:
X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model

model =SVR()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

print('y_pred is{} '.format(y_pred))
print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))

y_pred is[0.33798702 0.65193273 0.20250435 ... 0.6220346  0.40269238 0.51674984] 
MAE is 0.07123277013424491
MAPE is 0.39797205642337513
MSE is 0.008632638351870796
R2 score is 0.8200626729159709
RMSE score is 0.09291199250834521


In [ ]:

from sklearn.model_selection import GridSearchCV

# Define the SVR model
model = SVR()

# Define the hyperparameter grid for tuning
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'gamma': [0.1, 0.01, 0.001],
    'epsilon': [0.1, 0.01, 0.001]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Use the best model for prediction
y_pred = best_model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("RMSE:", rmse)
print("R2 Score:", r2)


In [75]:
def k_fold_cv(X,y,fold=10):
    score_lr = cross_val_score(LinearRegression(),X,y,cv = fold)
    score_ls = cross_val_score(Lasso(),X,y,cv = fold)
    score_rd = cross_val_score(Ridge(),X,y,cv = fold)
    score_dtr = cross_val_score(DecisionTreeRegressor(),X,y,cv = fold)
    score_svr = cross_val_score(SVR(),X,y,cv = fold)
    score_knn = cross_val_score(KNeighborsRegressor(),X,y,cv = fold)
    score_rf = cross_val_score(RandomForestRegressor(),X,y,cv = fold)
    score_ab = cross_val_score(AdaBoostRegressor(),X,y,cv = fold)
    score_gb = cross_val_score(GradientBoostingRegressor(),X,y,cv = fold)
    score_xb = cross_val_score(XGBRegressor(),X,y,cv = fold)
    
    model_name = ["Linear Regression","Lasso","Ridge","DTR","SVR","KNN","Random Forest Regressor",
                  "Ada Boost Regressor","Gradient Boosting Regressor","xg Boosting Regressor"]
    scores = [score_lr,score_ls,score_rd,score_dtr,score_svr,score_knn,
             score_rf,score_ab,score_gb,score_xb]
    result = []
    for i in range(len(model_name)):
        score_mean = np.mean(scores[i])
        score_std = np.std(scores[i])
        m_name = model_name[i]
        tmp = [m_name,score_mean,score_std]
        result.append(tmp)
    k_fold_df = pd.DataFrame(result,columns = ["Model Name","CV Accuracy","CV Standard Deviation"])
    return k_fold_df.sort_values("CV Accuracy",ascending =  False)
    

In [76]:
k_fold_cv(pca_df.drop("product_wg_ton",axis = 1),pca_df["product_wg_ton"])

,Model Name,CV Accuracy,CV Standard Deviation
4,SVR,0.83,0.01
9,xg Boosting Regressor,0.78,0.01
6,Random Forest Regressor,0.74,0.01
8,Gradient Boosting Regressor,0.73,0.01
2,Ridge,0.68,0.01
0,Linear Regression,0.68,0.01
7,Ada Boost Regressor,0.62,0.01
5,KNN,0.59,0.01
3,DTR,0.46,0.02
1,Lasso,-0.00,0.00


In [ ]:
#Tuning with Hyper parameters

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pca_df.drop('product_wg_ton' ,axis=1), pca_df['product_wg_ton'], test_size=0.2, random_state=42)
# Create a LinearRegression model


model= RandomForestRegressor(random_state=0, max_depth=9, min_samples_split=5, min_samples_leaf=2, 
                           max_features='auto')


# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
rmse = math.sqrt(mse)
print('y_pred is{} '.format(y_pred))
print('MAE is {}'.format(mae))
print('MAPE is {}'.format(mape))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('RMSE score is {}'.format(rmse))